In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!pip install -q catboost
!pip install -q lightgbm
!pip install -q xgboost
!pip install -q mlxtend
!pip install -q dask[dataframe]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 4.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import f1_score
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv("train_contest.csv", encoding='utf-8')

data

,premium,name,department,has_test,response_letter_required,area,type,address,response_url,sort_point_distance,...,working_time_intervals,working_time_modes,accept_temporary,description,experience,key_skills,specializations,region,immediate_redirect_url,mean_salary
0,False,Агент по недвижимости,NaN,False,False,"{'id': '2', 'name': 'Санкт-Петербург', 'url': ...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Санкт-Петербург', 'street': 'Невский...",NaN,NaN,...,[],[],False,"<p>Мы ищем людей, которым не достаточно того, ...","{'id': 'noExperience', 'name': 'Нет опыта'}","[{'name': 'Жилая недвижимость'}, {'name': 'Акт...","[{'id': '20.20', 'name': 'Агент', 'profarea_id...",Санкт-Петербург,NaN,125000.0
1,False,IT-специалист/ техник,NaN,False,False,"{'id': '1130', 'name': 'Братск', 'url': 'https...","{'id': 'open', 'name': 'Открытая'}",NaN,NaN,NaN,...,[],[],False,<ul> </ul> <p><em><strong>Навыки</strong></em>...,"{'id': 'between1And3', 'name': 'От 1 года до 3...",[],"[{'id': '1.172', 'name': 'Начальный уровень, М...",Иркутская область,NaN,50000.0
2,False,Старший/ведущий инженер-программист,NaN,False,False,"{'id': '76', 'name': 'Ростов-на-Дону', 'url': ...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Ростов-на-Дону', 'street': '60К-9, 1...",NaN,NaN,...,[],[],False,<strong>Обязанности:</strong> <ul> <li>Разрабо...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': 'Linux'}, {'name': 'Bash'}, {'name':...","[{'id': '1.221', 'name': 'Программирование, Ра...",Ростовская область,NaN,80000.0
3,False,Эксперт / методист приемной комиссии,NaN,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Москва', 'street': '1-я Миусская ули...",NaN,NaN,...,[],[],False,<p>Департамент информационных технологий РХТУ ...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': '1С программирование'}, {'name': 'Об...","[{'id': '3.150', 'name': 'Менеджмент продукта ...",Москва,NaN,120000.0
4,False,Табельщица/табельщик,NaN,False,False,"{'id': '1613', 'name': 'Энгельс', 'url': 'http...","{'id': 'open', 'name': 'Открытая'}","{'city': None, 'street': None, 'building': Non...",NaN,NaN,...,[],[],False,<p><strong>Обязанности:</strong></p> <ul> <li>...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': 'Составление отчетности'}, {'name': ...","[{'id': '18.142', 'name': 'Машиностроение', 'p...",Саратовская область,NaN,15000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30067,False,Руководитель отдела персонала,NaN,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Москва', 'street': 'Ленинградский пр...",NaN,NaN,...,[],[],False,<strong>Обязанности:</strong> <ul> <li>Управле...,"{'id': 'moreThan6', 'name': 'Более 6 лет'}","[{'name': 'Подбор персонала'}, {'name': 'Адапт...","[{'id': '9.738', 'name': 'Другое', 'profarea_i...",Москва,NaN,190000.0
30068,False,Специалист по развитию и обучению персонала,NaN,False,False,"{'id': '1641', 'name': 'Набережные Челны', 'ur...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Набережные Челны', 'street': 'проспе...",NaN,NaN,...,[],[],False,<p>Группа компаний &quot;Бринэкс&quot; приглаш...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': 'Управление временем'}, {'name': 'Ор...","[{'id': '6.254', 'name': 'Рекрутмент', 'profar...",Республика Татарстан,NaN,45000.0
30069,False,Инженер-программист,NaN,False,False,"{'id': '1399', 'name': 'Магнитогорск', 'url': ...","{'id': 'open', 'name': 'Открытая'}",NaN,NaN,NaN,...,[],[],False,<strong>Обязанности:</strong> <ul> <li>Разрабо...,"{'id': 'between1And3', 'name': 'От 1 года до 3...","[{'name': 'SQL'}, {'name': 'Работа с базами да...","[{'id': '1.221', 'name': 'Программирование, Ра...",Челябинская область,NaN,75000.0
30070,False,Куратор проекта,"{'id': '1947314-1947314-ancor', 'name': 'ANCOR'}",False,False,"{'id': '3', 'name': 'Екатеринбург', 'url': 'ht...","{'id': 'open', 'name': 'Откр

In [ ]:
!pip install optuna
!pip install LightGBM

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.0 MB/s eta 0:00:00


In [ ]:
data.premium.value_counts()

,count
premium,
False,29970
True,102


In [ ]:
import ast
def preprocess_data(salary):

    def ret(x):
      if pd.isna(x):
        return None
      else:
        try:
          return ast.literal_eval(x)[0]['name']
        except:
          return 'empty'


    # salary ~ X_train

    salary.groupby('premium')['name'].count()
    salary['name'].unique
    salary = salary.drop(columns=['name'])
    salary['department'] = salary['department'].apply(
        lambda x: None if pd.isna(x) else ast.literal_eval(x)['name']
    )
    salary['area'] = salary['area'].apply(
        lambda x: None if pd.isna(x) else ast.literal_eval(x)['id']
    )
    salary['type'] = salary['type'].apply(
        lambda x: None if pd.isna(x) else ast.literal_eval(x)['id']
    )
    salary['address'] = salary['address'].apply(
        lambda x: None if pd.isna(x) else ast.literal_eval(x)['city']
    )
    salary = salary.drop(columns=['response_url'])
    salary = salary.drop(columns=['sort_point_distance'])
    salary = salary.drop(columns=['published_at'])
    salary = salary.drop(columns=['insider_interview'])
    salary = salary.drop(columns=['relations'])
    salary['employer'] = salary['employer'].apply(
        lambda x: None if pd.isna(x) else ast.literal_eval(x)['name']
    )
    salary = salary.drop(columns=['snippet'])
    salary = salary.drop(columns=['contacts'])
    salary['schedule'] = salary['schedule'].apply(
        lambda x: None if pd.isna(x) else ast.literal_eval(x)['id']
    )
    salary = salary.drop(columns=['working_days'])
    salary = salary.drop(columns=['working_time_intervals'])
    salary = salary.drop(columns=['working_time_modes'])
    salary = salary.drop(columns=['description'])
    salary['experience'] = salary['experience'].apply(
        lambda x: None if pd.isna(x) else ast.literal_eval(x)['id']
    )
    salary['specializations'] = salary['specializations'].apply(
        lambda x: None if pd.isna(x) else ast.literal_eval(x)[0]['profarea_id']
    )
    salary['key_skills'] = salary['key_skills'].apply(
        lambda x: ret(x)
    )
    salary = salary.drop(columns=['key_skills'])
    # salary = salary.drop(columns=['specializations'])
    salary = salary.drop(columns=['alternate_url'])
    salary = salary.drop(columns=['url'])
    salary = salary.drop(columns=['immediate_redirect_url'])
    salary = salary.drop(columns=['created_at'])

    if labels is not None:
      salary['label'] = labels
      salary = salary[~salary['address'].isnull()]
      salary = salary[~salary['department'].isnull()]
      return salary.drop(columns='label'), salary['label']
    else:
      salary = salary.drop(columns=['address', 'department'])
      dfdf = pd.get_dummies(salary)
      return dfdf
#     scaler = StandardScaler()
#     scaled = scaler.fit_transform(df_encoded)
#     final = scaler.transform(scaled)

    return df_encoded
X = preprocess_data(data)

In [ ]:
X

,premium,has_test,response_letter_required,archived,accept_temporary,mean_salary,area_1,area_10,area_1011,area_1013,...,region_Тюменская область,region_Удмуртская Республика,region_Ульяновская область,region_Хабаровский край,region_Ханты-Мансийский АО - Югра,region_Челябинская область,region_Чеченская республика,region_Чувашская Республика,region_Ямало-Ненецкий АО,region_Ярославская область
0,False,False,False,False,False,125000.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,50000.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,80000.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,120000.0,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,15000.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30067,False,False,False,False,False,190000.0,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
30068,False,False,False,False,False,45000.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
30069,False,False,False,False,False,75000.0,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
30070,False,False,False,False,True,30000.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
y = X.mean_salary
X = X.drop('mean_salary', axis = 1)
y

,mean_salary
0,125000.0
1,50000.0
2,80000.0
3,120000.0
4,15000.0
...,...
30067,190000.0
30068,45000.0
30069,75000.0
30070,30000.0


In [ ]:
X_k = pd.read_csv('for_prediction.csv')
X_k = preprocess_data(X_k)


missing_cols = set(X.columns) - set(X_k.columns)
for col in missing_cols:
    X_k[col] = 0


X_k = X_k[X.columns]

In [ ]:
X_k

,premium,has_test,response_letter_required,archived,accept_temporary,area_1,area_10,area_1011,area_1013,area_1014,...,region_Тюменская область,region_Удмуртская Республика,region_Ульяновская область,region_Хабаровский край,region_Ханты-Мансийский АО - Югра,region_Челябинская область,region_Чеченская республика,region_Чувашская Республика,region_Ямало-Ненецкий АО,region_Ярославская область
0,False,False,False,False,False,True,False,False,0,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,0,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,0,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,0,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,True,False,False,0,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7513,False,False,False,False,False,False,False,False,0,False,...,False,False,False,False,False,False,False,False,False,False
7514,False,False,False,False,False,True,False,False,0,False,...,False,False,False,False,False,False,False,False,False,False
7515,False,False,False,False,False,False,False,False,0,False,...,False,False,False,False,False,False,False,False,False,False
7516,False,False,False,False,False,False,False,False,0,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
!pip install LightGBM

In [ ]:
from sklearn.metrics import mean_absolute_error
import optuna

X = np.array(X)

def objective(trial):
    # set up the parameters to be optimized
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    param = {
        'metric': 'mae',
        'n_estimators': trial.suggest_int('n_estimators', 50, 2000),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-4, 10.0,log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-4, 10.0,log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2,log=True),
        'max_depth': trial.suggest_int('max_depth', 4, 200),
        'num_leaves': trial.suggest_int('num_leaves', 40, 500),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 500),
        'cat_smooth': trial.suggest_int('cat_smooth', 1, 300),
        'force_col_wise':True
    }

    # create the LightGBM regressor with the optimized parameters
    model = LGBMRegressor(**param, n_jobs=-1, verbosity=-1)

    # perform cross-validation using the optimized LightGBM regressor
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    return mae


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

[I 2025-02-20 17:56:27,313] A new study created in memory with name: no-name-fcd67990-f14b-41b4-9aaf-7e4fa503e10b
[I 2025-02-20 17:56:39,641] Trial 0 finished with value: 41476.695273555546 and parameters: {'n_estimators': 1249, 'reg_alpha': 0.4710807319058836, 'reg_lambda': 6.340782789700361, 'colsample_bytree': 0.5974298389885586, 'subsample': 0.7355752861612874, 'learning_rate': 0.021814564979516792, 'max_depth': 169, 'num_leaves': 416, 'min_child_samples': 149, 'cat_smooth': 288}. Best is trial 0 with value: 41476.695273555546.
[I 2025-02-20 17:56:47,920] Trial 1 finished with value: 42933.232503902684 and parameters: {'n_estimators': 56, 'reg_alpha': 0.24903395448829785, 'reg_lambda': 0.024797519726836265, 'colsample_bytree': 0.8986433603544728, 'subsample': 0.7555797928446104, 'learning_rate': 0.13148669125518442, 'max_depth': 13, 'num_leaves': 423, 'min_child_samples': 182, 'cat_smooth': 6}. Best is trial 0 with value: 41476.695273555546.
[I 2025-02-20 17:56:59,869] Trial 2 fini

In [ ]:
param = {'n_estimators': 907, 'reg_alpha': 0.897634475540116, 'reg_lambda': 0.00010008582979898868, 'colsample_bytree': 0.7810612795170259, 'subsample': 0.5532319695705105, 'learning_rate': 0.05079860356992049, 'max_depth': 153, 'num_leaves': 485, 'min_child_samples': 1, 'cat_smooth': 156}

In [ ]:
model = LGBMRegressor(**param)
model.fit(np.array(X), y)

pred = model.predict(np.array(X_k))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010369 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 27762
[LightGBM] [Info] Number of data points in the train set: 30072, number of used features: 13881
[LightGBM] [Info] Start training from score 97728.250914
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

In [ ]:
pred

array([ 30591.6672536 ,  42283.88794263,  33631.54685312, ...,
        98324.2431604 , 116470.5289109 , 131003.99289735])

In [ ]:
my_data = pd.DataFrame({'Id':range(len(pred)), 'predicted': pred})
my_data.set_index('Id')
my_data

,Id,predicted
0,0,39644.793032
1,1,45849.569729
2,2,32583.079069
3,3,144228.660710
4,4,94121.892414
...,...,...
7513,7513,38327.420143
7514,7514,66957.471789
7515,7515,90824.969612
7516,7516,114158.565000


In [ ]:
my_data.to_csv('sub.csv', index = False)